Load the required packages

In [ ]:
import sample_jsonl
import json
import topic_filtering
import time_series
import visualisation
import pandas as pd
import os
import os.path as op

Load a sample of the dataset from GitHub

In [ ]:
file_path = 'data/yt_metadata_en.jsonl.gz'
sample_size = 1000000
random_seed = 42

df_vd_sampled = sample_jsonl.random_sample_jsonl_rows(file_path, sample_size, random_seed)

### Load the dataset from Viva's work (only Education videos)

Here: each video has the category of its channel since all videos were classified as Education

In [2]:
df_vd_sampled = pd.read_csv('data/updated_education_videos.csv')
sample_size = 0

/var/folders/1y/f431dfd55sgcrd9ymcz950th0000gn/T/ipykernel_8604/1497819074.py:1: DtypeWarning: Columns (0,5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vd_sampled = pd.read_csv('data/updated_education_videos.csv')


Load the events

In [ ]:
df_vd_sampled = pd.read_csv('data/updated_education_videos.csv')
sample_size = 0

with open(op.join('data','raw',"events.json"), "r") as file:
    events = json.load(file)

frequencies = ["ME", "W"]

for freq in frequencies:
    for topic, data in events.items():
        url = data['url']
        events_dict = data['events']
        text = topic_filtering.extract_text_from_wikipedia(url)
        keywords = topic_filtering.find_keywords(text)

        weights = {'title': 2, 'description': 0.5, 'tags': 1}
        mask = df_vd_sampled.apply(lambda row: topic_filtering.meets_keyword_requirements(row, keywords, weights), axis=1)
        df_vd_filtered = df_vd_sampled[mask]

        df_ts_s = time_series.create_time_slots(df_vd_filtered, freq)
        visualisation.plot_time_series(df_ts_s,topic,sample_size, freq)
        visualisation.plot_time_series(df_ts_s,topic,sample_size, freq, events_dict)
        visualisation.plot_cross_correlation(df_ts_s, topic, sample_size)